In [5]:
# Cell 1: Setup and Imports
import os
import openai
import json
import re
from datetime import datetime

# Set your OpenAI API key here or ensure OPENAI_API_KEY is set in your environment
OPENAI_API_KEY = "sk-…"  
openai.api_key = OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

class ScamDetector:
    def __init__(self):
        # Pass the key explicitly to ensure the client picks it up
        self.client = openai.Client(api_key=openai.api_key)

    def get_scam_prompt(self, language="english"):
        """Return the system prompt for the specified language."""
        prompts = {
            "english": """
You are an expert scam call detection system. Analyze this phone conversation transcript for scam indicators.

COMMON SCAM PATTERNS TO DETECT:
1. URGENCY: "act now", "limited time", "expires today"
2. FEAR TACTICS: threats, consequences, "you're in trouble"
3. PERSONAL INFO REQUESTS: SSN, passwords, banking details, codes
4. PAYMENT DEMANDS: wire transfers, gift cards, immediate payment
5. IMPERSONATION: claiming to be bank, IRS, tech support, family
6. TOO GOOD TO BE TRUE: lottery wins, free money, guaranteed returns
7. SECRECY: "don't tell anyone", "keep this between us"
8. PRESSURE: not allowing time to think or verify

Respond ONLY with valid JSON:
{
  "risk_level": "SAFE|SUSPICIOUS|SCAM",
  "confidence": 0.85,
  "detected_patterns": ["list of specific patterns found"],
  "scam_type": "tech_support|impersonation|financial|romance|phishing|unknown",
  "reasoning": "brief explanation"
}
""",
        }
        return prompts.get(language, prompts["english"])

    def analyze_transcript(self, transcript: str, language="english") -> dict:
        """Analyze a transcript and return a dict with scam analysis."""
        try:
            response = self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": self.get_scam_prompt(language)},
                    {"role": "user",   "content": f"TRANSCRIPT: {transcript}"}
                ],
                temperature=0.1,
                max_tokens=400
            )
            content = response.choices[0].message.content
            return json.loads(content)

        except Exception as e:
            # On error, return a safe default structure
            return {
                "risk_level": "UNKNOWN",
                "confidence": 0.0,
                "detected_patterns": [],
                "scam_type": "unknown",
                "reasoning": f"Analysis failed: {e}"
            }


# Cell 2: Testing the ScamDetector
detector = ScamDetector()

test_transcripts = [
    {
        "text": "Hello, this is John from Microsoft technical support. We've detected a virus on your computer. I need you to give me remote access immediately to fix this urgent problem.",
        "language": "english"
    },
    {
        "text": "Hi mom, it's me your grandson. I'm in jail and need you to wire $2000 immediately. Don't tell my parents, this is embarrassing.",
        "language": "english"
    },
    {
        "text": "Good morning, how are you today? I'm calling to remind you about your appointment next Tuesday.",
        "language": "english"
    }
]

print("🔍 Testing Scam Detection:")
print("=" * 60)
for i, test in enumerate(test_transcripts, start=1):
    print(f"\nTest {i}:")
    print(f"Transcript: {test['text']}")
    result = detector.analyze_transcript(test['text'], test['language'])
    print(f"🚨 Risk Level: {result['risk_level']}")
    print(f"📊 Confidence: {result['confidence']:.2f}")
    print(f"🎯 Scam Type: {result['scam_type']}")
    print(f"⚠️ Patterns: {', '.join(result['detected_patterns'])}")
    print(f"💭 Reasoning: {result['reasoning']}")
    print("-" * 40)


🔍 Testing Scam Detection:

Test 1:
Transcript: Hello, this is John from Microsoft technical support. We've detected a virus on your computer. I need you to give me remote access immediately to fix this urgent problem.
🚨 Risk Level: UNKNOWN
📊 Confidence: 0.00
🎯 Scam Type: unknown
⚠️ Patterns: 
💭 Reasoning: Analysis failed: 'ascii' codec can't encode character '\u2026' in position 10: ordinal not in range(128)
----------------------------------------

Test 2:
Transcript: Hi mom, it's me your grandson. I'm in jail and need you to wire $2000 immediately. Don't tell my parents, this is embarrassing.
🚨 Risk Level: UNKNOWN
📊 Confidence: 0.00
🎯 Scam Type: unknown
⚠️ Patterns: 
💭 Reasoning: Analysis failed: 'ascii' codec can't encode character '\u2026' in position 10: ordinal not in range(128)
----------------------------------------

Test 3:
Transcript: Good morning, how are you today? I'm calling to remind you about your appointment next Tuesday.
🚨 Risk Level: UNKNOWN
📊 Confidence: 0.00
🎯 Sca